## Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os
from utils import read_file, get_fs_data, get_fs_data_query,import_lookup_dictionary, update_field_from_dictionary, 

### Permit Data Engineering

In [4]:
# county permit data
dfPlacerPermit = read_file("data\PermitData_Placer_040124.csv")
dfCSLTPermit   = read_file("data\PermitData_CSLT_040224.csv")
dfElDoPermit   = read_file("data\PermitData_ElDorado_040124.csv")
# trpa permit table
permitTable = "https://maps.trpa.org/server/rest/services/Permit_Records/MapServer/1"
dfTRPAPermit = get_fs_data(permitTable)

In [ ]:
# lookup list of permit types
projectLookup = get

In [5]:
dfTRPAPermit

,APN,Accela_ID,Key_One,Key_Two,Key_Three,Short_Notes,Detailed_Description,Record_Status,Accela_CAPType_Name,File_Date,Assigned_To_Staff,OBJECTID,ESRI_OID
0,023-641-01,20010557SA,01HIS,0,01150,FILE ATTACHED alg,Site Assessment.,Project Completed,Building/ERS/Assessments/Site Assessment,1001376000000,OLD FILE,6592,1
1,001-020-16,20010491STD,01HIS,0,07162,FILE ATTACHED alg,Single family dwelling (New) Separate erosion ...,Project Completed,Building/ERS/Permits/Res Dwelling,995414400000,Jeanne McNamara,11057,1
2,090-046-005,ERSP2012-0916,12HIS,0,01541,STACKS 5/16/2018.mm,Re-issuance of project ERSP2010-0769.\nPERMIT ...,Acknowledged,Building/ERS/Permits/Commercial,1344470400000,OLD FILE,85571,1
3,085-222-005,ERSP2016-1256,16HIS,0,02452,stacks 5/24/18 kw,PIER EXTENSION. \nNOTE: This permit relocates ...,Final Inspection Complete,Building/ERS/Permits/Shore-Lakezone,1480636800000,Tiffany Good,114695,1
4,122-530-00,ERSP2020-1181,20HIS,0,01642,FILE ATTACHED,Interior remodel which combines units 25 & 26....,Project Complete,Building/ERS/Permits/Res Dwelling,1594944000000,Bridget Cornell,135508,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
222995,1318-15-818-003,20000830STD,00HIS,0,05980,Project Box 70 - RETURNED TO BOX 70 8/12/10,1st PLAN REVISION TO 19980183STD.\nTime shari...,Project Completed,Building/ERS/Permits/Commercial,975888000000,Lyn Barnett,3852,1
222996,027-090-16,20000009STD,00HIS,0,06048,scanning contractor 2/20/24,Antenna Addition.\nHEARINGS OFFICER APPROVED 2...,Project Completed,Building/ERS/Permits/Rec-Public Service,947548800000,OLD FILE,4068,1
222997,007-100-01,20010035STD,00HIS,0,06049,Located in Box 1 of 4 - 134 Kahle,Transfer of 424 SF of CFA to Receiving Parcel ...,Project Completed,Building/ERS/Admin/Transfer of Development,971913600000,OLD FILE,4069,1
222998,1318-22-002-003,20010035STD,00HIS,0,06049,Located in Box 1 of 4 - 134 Kahle,Transfer of 424 SF of CFA to Receiving Parcel ...,Project Completed,Building/ERS/Admin/Transfer of Development,971913600000,OLD FILE,4070,1


In [ ]:
# Placer Data - MOU
folder_path = 'F:\Research and Analysis\Local Jurisdiction MOU data collection\Placer MOU Files\Placer'
dfs = []

# Loop through the files in the folder and identify CSV files
for file_name in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Read the CSV file into a DataFrame and append to the list
    df = pd.read_excel(file_path)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Display the resulting DataFrame
print(final_df)

### Annual Report Data Engineering

#### Permit Data